In [1]:
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception unknown locale: UTF-8.


In [2]:
resource_name = "Demo_RS"
workspace_name = "Demo_WS"
subscriptionID = "***********************" # Please enter your subscription id here
aml_compute_target = "demo-cluster" #ALL SMALL LETTER, NO underscore, 16ws long only.
experiment_name= 'demo_expirement'

In [3]:
# Now create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US")
    ws.write_config('.azureml')


Deploying AppInsights with name demowsinsightsc1e0bdcb8d.
Deployed AppInsights with name demowsinsightsc1e0bdcb8d. Took 6.58 seconds.
Deploying KeyVault with name demowskeyvaultc3bddf8ae0.
Deploying StorageAccount with name demowsstoragedaccd5ca045.
Deployed KeyVault with name demowskeyvaultc3bddf8ae0. Took 22.8 seconds.
Deployed StorageAccount with name demowsstoragedaccd5ca045. Took 29.61 seconds.
Deploying Workspace with name Demo_WS.
Deployed Workspace with name Demo_WS. Took 29.59 seconds.


In [4]:
Workspace.from_config()

Workspace.create(name='Demo_WS', subscription_id='ff34ca9f-21a3-4bc6-b226-c2fccc42b164', resource_group='Demo_RS')

In [5]:
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")


creating new compute target : demo-cluster
Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached now


In [6]:
# Create Experiment
exp = Experiment(ws,experiment_name)

In [ ]:
# Create environment to execute your code
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
config=ScriptRunConfig(source_directory="./",script="mytrain.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)


In [8]:
config=ScriptRunConfig(source_directory="./",script="mytrain.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7])
execution=exp.submit(config)



In [9]:
config=ScriptRunConfig(source_directory="./",script="mytrain_log.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7])
execution=exp.submit(config)

In [10]:
#Register model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./diabeticmodel.pkl",model_name="first_ml_model")

Registering model first_ml_model


In [12]:
env.environment_variables

{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}

In [14]:
model = Model(ws,"first_ml_model")

In [15]:
myenv=Environment(name="demo-env")
conda_packages = ['numpy']
pip_packages = ['azureml-sdk','azureml-defaults','scikit-learn']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.6.2')
myenv.python.conda_dependencies=mycondaenv
myenv.register(workspace=ws)

inference_config = InferenceConfig(entry_script='score.py',source_directory='.',environment=myenv)


In [17]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [18]:
# Deploying the model
service = Model.deploy(ws,"demomodel", #This is endpoint name
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           #deployment_target=predenv,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-27 12:14:21+05:30 Creating Container Registry if not exists.
2021-05-27 12:14:21+05:30 Registering the environment.
2021-05-27 12:14:23+05:30 Building image..
2021-05-27 12:21:39+05:30 Generating deployment configuration.
2021-05-27 12:21:40+05:30 Submitting deployment to compute.
2021-05-27 12:21:45+05:30 Checking the status of deployment demomodel..
2021-05-27 12:23:33+05:30 Checking the status of inference endpoint demomodel.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://756dbf5d-b758-48d3-ac85-93fb969239ae.eastus.azurecontainer.io/score


In [19]:
# Prepare your test data
import pandas as pd
import json
data = pd.read_csv('test.csv')
xt = data.values.tolist()
test = json.dumps({"data":xt})

In [ ]:
test

In [ ]:
from azureml.core import Webservice
service = Webservice(workspace=ws, name="demomodel")
print(service.scoring_uri)
print(service.swagger_uri)
prediction = service.run(input_data=test)
prediction
# http://756dbf5d-b758-48d3-ac85-93fb969239ae.eastus.azurecontainer.io/score
# http://756dbf5d-b758-48d3-ac85-93fb969239ae.eastus.azurecontainer.io/swagger.json

In [22]:
xt = data.values.tolist()
xt[0:2]
tworecords = json.dumps({"data":xt[0:2]})
tworecords

'{"data": [[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0], [1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.35100000000000003, 31.0]]}'

In [23]:
prediction = service.run(input_data=tworecords)
prediction

[1, 0]

In [24]:
# Real time scoring using Rest Call
import requests
headers = {'content-Type':'application/json'}

response = requests.post(service.scoring_uri, test, headers=headers)
response.text

'[1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,